In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2 
import numpy as np
import os # for filepaths
from matplotlib import pyplot as plt
import time
import mediapipe as mp # Para utilizar el modelo holistic

mp_holistic = mp.solutions.holistic # Holistic Model, es un modelo que detecta landmarks faciales y corporales.
mp_drawing = mp.solutions.drawing_utils # Utils para dibujar los landmarks.

def mediapipe_detection(image, model): # Función que recibe una imagen (frame)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion 
    image.flags.writeable = False # image status
    results = model.process(image) #make pred
    image.flags.writeable = True # img status writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conv  
    return image, results 

# Función para dibujar landmarks, recibe una imagen(frame) y los resultados del modelo (holistic)
def draw_landmarks(image, results):
# ROSTRO
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image, # Se pasa el frame
            results.face_landmarks, # Se verifica si hay resultados correspondientes a la cara.
            mp_holistic.FACEMESH_TESSELATION, # Se pasan los nodos.
            mp_drawing.DrawingSpec(color=(209, 12, 163), thickness=1, circle_radius=1), # Nodos
            mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=1, circle_radius=1) # Líneas
        )
#POSE (BRAZOS, PECHO) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )
# MANO IZQUIERDA
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )
# MANO DERECHA
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('trainTest') 

# Actions that we try to detect
actions = np.array(['hola', 'mi_nombre_es', 'como_estas', 'chao', 'buenas_noches', 'INSOR', 'por_favor', 'parado'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

label_map = {label:num for num, label in enumerate(actions)}
label_map

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

2024-11-13 01:01:22.712187: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-13 01:01:22.712232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
np.array(labels).shape #50*11

(400,)

In [104]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [105]:
X = np.array(sequences)
X.shape # (50 Videos * 11 Labels, 30 frames cada uno, 1662 keypoints)

(400, 30, 1662)

In [106]:
y = to_categorical(labels).astype(int) # Convertir enteros a categorical info
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir = log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30,1662))) # We need to return the sequences if we have continus lstm layers
model.add(LSTM(128,return_sequences=True, activation="relu"))
model.add(LSTM(64,return_sequences=False, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(actions.shape[0], activation="softmax"))

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])
#model.fit(X_train, y_train, epochs=1600, callbacks=[tb_callback])

2024-11-13 01:01:45.115196: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2024-11-13 01:01:45.115249: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2024-11-13 01:01:45.116067: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/saur/anaconda3/envs/Lumiere/lib/python3.8/site-packages/cv2/../../lib64:
2024-11-13 01:01:45.116164: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2024-11-13 01:01:45.130296: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-11-13 01:01:45.130646: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-13 01:01:45.130694: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
Total params: 596,840
Trainable params: 596,840
Non-trainable params: 0
__________________________________________________

In [5]:
res = model.predict(X_test)


NameError: name 'X_test' is not defined

In [6]:
actions[np.argmax(y_test[4])]


NameError: name 'y_test' is not defined

In [112]:
model.save("tryfewer.h5")

In [7]:
model.load_weights("tryfewer.h5")


In [8]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [9]:
yhat = model.predict(X_test)


NameError: name 'X_test' is not defined

In [115]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [116]:
multilabel_confusion_matrix(ytrue, yhat)


array([[[18,  0],
        [ 1,  1]],

       [[14,  0],
        [ 0,  6]],

       [[18,  0],
        [ 0,  2]],

       [[17,  0],
        [ 0,  3]],

       [[18,  0],
        [ 0,  2]],

       [[19,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  1]],

       [[16,  1],
        [ 0,  3]]])

In [117]:
accuracy_score(ytrue, yhat)


0.95

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245), (117,245,16),(117,245,16),(117,245,16),(117,245,16),(117,245,16),(117,245,16),(117,245,16),(117,245,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for idx, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + idx*40), (int(prob*100), 90 + idx*40), colors[idx], -1)
        cv2.putText(output_frame, f'{actions[idx]}: {prob:.2f}', (5, 85 + idx*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-30:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot o

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

2024-11-13 01:02:19.318247: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-11-13 01:02:19.318941: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3293770000 Hz


parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
por_favor
<class 'mediapipe.python.solution_base.SolutionOutputs'>
parado
<class 'mediapipe.python.solut

In [45]:
cap.release()
cv2.destroyAllWindows() #Stop all CASE